In [1]:
##### add python path #####
import sys
import os
PATH = os.getcwd()
for dir_idx, dir_name in enumerate(PATH.split('/')):
    if 'optimal_control' in dir_name.lower():
        PATH = '/'.join(PATH.split('/')[:(dir_idx+1)])
        break
if not PATH in sys.path:
    sys.path.append(PATH)
###########################
from env.quadruped.env import Env

from trajectory_planner import TrajectoryPlanner
from foot_step_planner import FootStepPlanner
from gait_scheduler import GaitScheduler
from NLP_solver import NLPSolver

from scipy.spatial.transform import Rotation
from copy import deepcopy
import numpy as np
import time

In [2]:
env = Env(enable_draw=True, base_fix=False)
num_leg = env.num_leg
time_step = env.time_step
base_inertia = deepcopy(env.model.inertia)
base_mass = env.model.mass
friction_coef = env.friction_coef
max_force = env.model.max_force

plan_time_step = 0.05
plan_time_horizon = 10

gs_args = dict()
gs_args['num_leg'] = num_leg
gs_args['gait_period'] = 1.0
gs_args['is_visualize'] = False
gait_scheduler = GaitScheduler(gs_args)

tp_args = dict()
tp_args['time_step'] = plan_time_step
tp_args['time_horizon'] = plan_time_horizon
tp_args['max_accel'] = 0.1
tp_args['max_ang_accel'] = 0.1
trajectory_planner = TrajectoryPlanner(tp_args)

fsp_args = dict()
fsp_args['num_leg'] = num_leg
fsp_args['time_horizon'] = plan_time_horizon
fsp_args['abduct_org_list'] = env.model.abduct_org_list
foot_step_planner = FootStepPlanner(fsp_args)

state_dim = 12
force_dim = 12
R_mat = np.eye(force_dim)*0.01 #001
Q_mat = np.eye(state_dim)
Qf_mat = np.eye(state_dim)*100.0
nlp_args = dict()
nlp_args['R_mat'] = R_mat
nlp_args['Q_mat'] = Q_mat
nlp_args['Qf_mat'] = Qf_mat
nlp_args['time_horizon'] = plan_time_horizon
nlp_args['time_step'] = plan_time_step
nlp_args['base_inertia'] = base_inertia
nlp_args['base_mass'] = base_mass
nlp_args['friction_coef'] = friction_coef
nlp_args['max_force'] = max_force
nlp_solver = NLPSolver(nlp_args)

In [3]:
env.reset()
nlp_solver.reset()
gait_scheduler.reset()
trajectory_planner.reset()
foot_step_planner.reset()

In [4]:
import imageio

class Renderer:
    def __init__(self, env):
        self.env = env
        self.RENDER_WIDTH, self.RENDER_HEIGHT = 800, 600

        self.view_matrix = env.pybullet_client.computeViewMatrixFromYawPitchRoll(
            cameraTargetPosition=env.camTargetPos,
            distance=env.camDist,
            yaw=env.camYaw,
            pitch=env.camPitch,
            roll=0,
            upAxisIndex=2)
        self.proj_matrix = env.pybullet_client.computeProjectionMatrixFOV(
            fov=60, aspect=float(self.RENDER_WIDTH)/self.RENDER_HEIGHT,
            nearVal=0.1, farVal=100.0)
        
        self.img_list = []
    
    def render(self):
        width, height, rgb_img, depth_img, seg_img = self.env.pybullet_client.getCameraImage(
            width=self.RENDER_WIDTH, 
            height=self.RENDER_HEIGHT,
            viewMatrix=self.view_matrix,
            projectionMatrix=self.proj_matrix)
        rgb_img = rgb_img[:,:,:3]
        bgr_img = rgb_img[:,:,::-1]
        self.img_list.append(rgb_img)

    def save_video(self, fps):
        output = "./result.mp4"
        writer = imageio.get_writer(output, fps=fps)
        for i, img in enumerate(self.img_list):
            sys.stdout.write("\rframe {0}".format(i))
            sys.stdout.flush()
            writer.append_data(img)
        print("\r\nFinalizing...")
        writer.close()
        
renderer = Renderer(env)

In [5]:
'''
gait_list = np.zeros((num_leg, 2)) # FR, FL, RR, RL
gait_list[0,0] = 0.5 # offset
gait_list[0,1] = 0.5 # duration
gait_list[2,0] = 0.0 # offset
gait_list[2,1] = 0.5 # duration
gait_scheduler.gait_list = deepcopy(gait_list)
gait_scheduler.update()
'''

vel_cmd = np.zeros(3)
vel_cmd[0] = 0.0
vel_cmd[1] = 0.0
ang_vel_cmd = np.zeros(3)
ang_vel_cmd[2] = 0.0
height_cmd = 0.3

In [6]:
cube = env.pybullet_client.loadURDF("cube_small.urdf")

collisionFilterGroup = 0x1
collisionFilterMask = 0x1
env.pybullet_client.setCollisionFilterGroupMask(env.planeId, -1, collisionFilterGroup, collisionFilterMask)
for i in range(env.pybullet_client.getNumJoints(env.model.sim_model)):
    env.pybullet_client.setCollisionFilterGroupMask(env.model.sim_model, i, collisionFilterGroup, collisionFilterMask)

collisionFilterGroup2 = 0x2
collisionFilterMask2 = 0x2 
for i in range(env.pybullet_client.getNumJoints(cube)):
    env.pybullet_client.setCollisionFilterGroupMask(cube, i, collisionFilterGroup2, collisionFilterMask2)

In [7]:
global_t = 0.0
mpc_cnt = 0
mpc_period = int(plan_time_step/time_step)
while global_t < 10.0:
    if mpc_cnt == 0:
        init_state_list = np.concatenate([env.model.com_pos, env.model.base_rpy, env.model.base_vel, env.model.base_ang_vel])
        init_foot_pos_list = deepcopy(env.model.foot_pos_list)
        
        #vel_cmd[:2] = 10.0*(np.zeros(2) - env.model.com_pos[:2])

        target_trajectory_list = trajectory_planner.get_target_trajectory_list(vel_cmd, ang_vel_cmd, height_cmd, init_state_list)
        target_trajectory_list[:,:2] = 0.0
        
        contact_list, step_info_list = gait_scheduler.get_contact_list(plan_time_step, plan_time_horizon)
        target_foot_step_list = foot_step_planner.get_target_foot_step_list(init_foot_pos_list, step_info_list, target_trajectory_list, vel_cmd, ang_vel_cmd, height_cmd)
        state_list, force_list = nlp_solver.get_solution(init_state_list, target_trajectory_list, target_foot_step_list, contact_list)

        #'''
        for i in range(plan_time_horizon):
            time.sleep(0.1)
            state = state_list[i]
            pos_base = state[:3]
            rpy_base = state[3:6]
            vel_base = state[6:9]
            ang_vel_base = state[9:]
            orn_base = env.pybullet_client.getQuaternionFromEuler(rpy_base)

            env.pybullet_client.resetBasePositionAndOrientation(cube, pos_base, orn_base)
            env.pybullet_client.resetBaseVelocity(cube, np.zeros(3), np.zeros(3))
            #print(pos_base, target_trajectory_list[i,:3])
        #'''
        print(state_list[:,:3])

    contact_list, gait_info_list = gait_scheduler.step(time_step)
    inter_weight = 1.0 - float(mpc_cnt)/mpc_period
    inter_force_list = inter_weight*force_list[0] + (1-inter_weight)*force_list[1]
    #print(inter_force_list)
    env.step(inter_force_list, contact_list, desired_foot_list=[])
    #time.sleep(time_step)
    renderer.render()

    mpc_cnt = np.mod(mpc_cnt + 1, mpc_period)
    global_t += time_step
    
renderer.save_video(int(1/time_step))
gait_scheduler.close()

[[-7.44125739e-03  6.92041475e-07  1.70706439e-01]
 [-8.42735252e-03  2.71212158e-06  1.39364458e-01]
 [-9.28264979e-03  8.36177375e-07  1.14433880e-01]
 [-1.03062365e-02 -6.14515266e-06  9.52117314e-02]
 [-1.18322296e-02 -1.94703063e-05  8.09950840e-02]
 [-1.40664994e-02 -4.03627978e-05  7.10811310e-02]
 [-1.69070610e-02 -6.99861328e-05  6.47650279e-02]
 [-2.00978538e-02 -1.09444851e-04  6.13378798e-02]
 [-2.36157827e-02 -1.59966693e-04  6.00871283e-02]
 [-2.73921455e-02 -2.19789139e-04  6.02953556e-02]]
[[-1.98603911e-03  5.51348579e-06  1.56846323e-01]
 [-7.63672631e-04  3.16523406e-06  1.40237387e-01]
 [-6.13426431e-04 -5.10638379e-06  1.29360569e-01]
 [-1.66119628e-03 -2.14402774e-05  1.23156576e-01]
 [-3.96563165e-03 -4.89805244e-05  1.20575459e-01]
 [-7.35562136e-03 -9.23350658e-05  1.20576396e-01]
 [-1.15437241e-02 -1.57422460e-04  1.22121535e-01]
 [-1.64906084e-02 -2.50151930e-04  1.24170827e-01]
 [-2.23840153e-02 -3.69446749e-04  1.25689237e-01]
 [-2.49178131e-02 -4.87351149e

[[-0.03625986 -0.00098628  0.26517873]
 [-0.03632078 -0.00100742  0.26790457]
 [-0.03583306 -0.00100728  0.27128218]
 [-0.03505883 -0.00099145  0.27497053]
 [-0.03417683 -0.00096474  0.2786592 ]
 [-0.0333067  -0.00093226  0.28209315]
 [-0.03253628 -0.00089549  0.28504186]
 [-0.03191677 -0.00085583  0.28730724]
 [-0.03146007 -0.00081439  0.28874387]
 [-0.03115977 -0.00077379  0.28925162]]
[[-0.03740656 -0.00100114  0.26639251]
 [-0.0374262  -0.00102093  0.26890766]
 [-0.03685956 -0.00101683  0.27209756]
 [-0.03599293 -0.00099558  0.27560939]
 [-0.03500794 -0.00096351  0.2791514 ]
 [-0.03403575 -0.00092297  0.2824576 ]
 [-0.03317244 -0.00087594  0.28529074]
 [-0.03247225 -0.00082368  0.28745963]
 [-0.03193499 -0.00076769  0.28881233]
 [-0.0315526  -0.00070866  0.2892246 ]]
[[-0.03841132 -0.00102735  0.26744262]
 [-0.03846301 -0.00103622  0.26970076]
 [-0.03793512 -0.00102711  0.27256734]
 [-0.03708181 -0.00100651  0.27572481]
 [-0.03607968 -0.00097751  0.27888706]
 [-0.03505547 -0.000942

[[-4.84628405e-02 -5.15530497e-04  2.75228027e-01]
 [-4.78598776e-02 -4.56743286e-04  2.76310833e-01]
 [-4.67992392e-02 -3.95022587e-04  2.77920415e-01]
 [-4.55080865e-02 -3.32110028e-04  2.79802584e-01]
 [-4.41383868e-02 -2.69629831e-04  2.81737758e-01]
 [-4.27937066e-02 -2.09303987e-04  2.83533565e-01]
 [-4.15440845e-02 -1.53207983e-04  2.85018956e-01]
 [-4.04256988e-02 -1.03536591e-04  2.86046405e-01]
 [-3.94429134e-02 -6.21155401e-05  2.86494138e-01]
 [-3.86073926e-02 -3.04261754e-05  2.86267571e-01]]
[[-4.86101110e-02 -4.64325160e-04  2.75369197e-01]
 [-4.79567868e-02 -4.09631854e-04  2.76418231e-01]
 [-4.68399657e-02 -3.53500493e-04  2.77991310e-01]
 [-4.54917819e-02 -2.97483937e-04  2.79835326e-01]
 [-4.40658533e-02 -2.43358915e-04  2.81731518e-01]
 [-4.26715958e-02 -1.93306538e-04  2.83484093e-01]
 [-4.13847815e-02 -1.49666427e-04  2.84920433e-01]
 [-4.02470904e-02 -1.14498109e-04  2.85893632e-01]
 [-3.92645823e-02 -8.95484477e-05  2.86286834e-01]
 [-3.84387052e-02 -7.70282165e

[[-5.03495958e-02 -1.69275548e-04  2.77207741e-01]
 [-4.99768047e-02 -1.47828523e-04  2.77873095e-01]
 [-4.93091481e-02 -1.18860594e-04  2.78799701e-01]
 [-4.84987459e-02 -8.60483693e-05  2.79798642e-01]
 [-4.76423989e-02 -5.71691475e-05  2.80724370e-01]
 [-4.68034074e-02 -3.71882959e-05  2.81481429e-01]
 [-4.60224545e-02 -2.40056735e-05  2.82023951e-01]
 [-4.53161457e-02 -1.35548510e-05  2.82340183e-01]
 [-4.46904814e-02 -6.59095605e-06  2.82445738e-01]
 [-4.41672652e-02 -4.14547824e-06  2.82341911e-01]]
[[-5.04746236e-02 -1.54136653e-04  2.77320052e-01]
 [-5.00979383e-02 -1.31035475e-04  2.77943065e-01]
 [-4.94309201e-02 -1.01969767e-04  2.78776586e-01]
 [-4.86214695e-02 -7.47107162e-05  2.79639533e-01]
 [-4.77634487e-02 -5.43166718e-05  2.80405227e-01]
 [-4.69216824e-02 -3.88412159e-05  2.80999483e-01]
 [-4.61339161e-02 -2.43575978e-05  2.81390511e-01]
 [-4.54168184e-02 -1.17128495e-05  2.81590790e-01]
 [-4.47771996e-02 -2.36242753e-06  2.81622470e-01]
 [-4.42370109e-02  1.93687466e

[[-4.96095892e-02 -8.39490458e-05  2.77468465e-01]
 [-4.90868190e-02 -7.28136643e-05  2.77842700e-01]
 [-4.83206110e-02 -5.69819893e-05  2.78357550e-01]
 [-4.74421860e-02 -3.79390734e-05  2.78887362e-01]
 [-4.65342919e-02 -1.71728623e-05  2.79347666e-01]
 [-4.56522363e-02  3.88370180e-06  2.79689881e-01]
 [-4.48294438e-02  2.38126338e-05  2.79897963e-01]
 [-4.40795065e-02  4.12696066e-05  2.79980393e-01]
 [-4.34063765e-02  5.50134805e-05  2.79953503e-01]
 [-4.28309851e-02  6.39633732e-05  2.79817528e-01]]
[[-4.95256039e-02 -8.05851459e-05  2.77459200e-01]
 [-4.90024961e-02 -7.05191403e-05  2.77830331e-01]
 [-4.82361349e-02 -5.52787762e-05  2.78342024e-01]
 [-4.73575689e-02 -3.65311857e-05  2.78869137e-01]
 [-4.64494357e-02 -1.58766677e-05  2.79326691e-01]
 [-4.55669584e-02  5.17478845e-06  2.79665306e-01]
 [-4.47435059e-02  2.52452253e-05  2.79868675e-01]
 [-4.39927933e-02  4.30523166e-05  2.79946089e-01]
 [-4.33187327e-02  5.72146848e-05  2.79915508e-01]
 [-4.27419883e-02  6.65031578e

[[-4.84132812e-02 -5.91277427e-05  2.77426321e-01]
 [-4.79200529e-02 -5.14762119e-05  2.77831095e-01]
 [-4.71823667e-02 -3.77081985e-05  2.78390980e-01]
 [-4.63313759e-02 -1.98218443e-05  2.78976808e-01]
 [-4.54494498e-02  3.88698335e-07  2.79499273e-01]
 [-4.45915710e-02  2.13129550e-05  2.79904686e-01]
 [-4.37908841e-02  4.14437238e-05  2.80172041e-01]
 [-4.30607290e-02  5.93133566e-05  2.80304932e-01]
 [-4.24049881e-02  7.34990311e-05  2.80314718e-01]
 [-4.18458949e-02  8.28783004e-05  2.80197043e-01]]


KeyboardInterrupt: 

In [8]:
renderer.save_video(int(1/time_step))

frame 0

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (800, 600) to (800, 608) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


frame 449
Finalizing...


In [11]:
def get_cross_product_matrix(vector):
    a, b, c = vector
    mat = np.array([[0, -c, b],
                   [c, 0, -a],
                   [-b, a, 0]])
    return mat

def eq_cons_func(x, args):
    init_state_list = args[0]
    target_foot_step_list = args[1]
    contact_list = args[2]
    delta_t = args[3]
    base_inertia = args[4]
    base_mass = args[5]
    num_leg = 4

    time_horizon = int(len(x)/24)
    state_list = x[:int(len(x)/2)].reshape((time_horizon, 12))
    force_list = x[int(len(x)/2):].reshape((time_horizon, 12))
    
    eq_cons = []
    state = init_state_list
    gravity = np.zeros(12)
    gravity[6:9] = delta_t*np.array([0, 0, -9.8])
    for time_idx in range(time_horizon):
        next_state = state_list[time_idx]
        force = (force_list[time_idx].reshape((4,3))*contact_list[time_idx].reshape(4,1)).ravel()

        yaw = state[5]
        pos_com = state[:3]
        yaw_mat = Rotation.from_euler('z', yaw, degrees=False).as_matrix()
        global_inertia = np.matmul(yaw_mat, np.matmul(base_inertia, yaw_mat.T))
        inv_global_inertia = np.linalg.inv(global_inertia)
        foot_step_list = target_foot_step_list[time_idx]

        A_mat = np.eye(12)
        A_mat[0:3, 6:9] = delta_t*np.eye(3)
        A_mat[3:6, 9:12] = delta_t*yaw_mat.T #delta_t*yaw_mat

        B_mat = np.zeros((12, 12))
        for leg_idx in range(num_leg):
            foot_step = foot_step_list[leg_idx] - pos_com
            B_mat[6:9, leg_idx*3:(leg_idx+1)*3] = np.eye(3)*(delta_t/base_mass)
            B_mat[9:12, leg_idx*3:(leg_idx+1)*3] = delta_t*np.matmul(inv_global_inertia, get_cross_product_matrix(foot_step))
        
        eq_cons.append((np.matmul(A_mat, state) + np.matmul(B_mat, force) + gravity - next_state).ravel())
        state = next_state

    eq_cons = np.concatenate(eq_cons)
    return eq_cons

def eq_cons_jacobian(x, args):
    init_state_list = args[0]
    target_foot_step_list = args[1]
    contact_list = args[2]
    delta_t = args[3]
    base_inertia = args[4]
    base_mass = args[5]
    num_leg = 4

    time_horizon = int(len(x)/24)
    state_list = x[:int(len(x)/2)].reshape((time_horizon, 12))
    force_list = x[int(len(x)/2):].reshape((time_horizon, 12))
    
    jacobian = np.zeros((12*time_horizon, len(x)))
    state = init_state_list
    for time_idx in range(time_horizon):        
        next_state = state_list[time_idx]
        force = (force_list[time_idx].reshape((4,3))*contact_list[time_idx].reshape(4,1)).ravel()

        yaw = state[5]
        pos_com = state[:3]
        ang_vel_base = state[9:12]
        yaw_mat = Rotation.from_euler('z', yaw, degrees=False).as_matrix()
        global_inertia = np.matmul(yaw_mat, np.matmul(base_inertia, yaw_mat.T))
        inv_global_inertia = np.linalg.inv(global_inertia)
        inv_base_inertia = np.linalg.inv(base_inertia)
        foot_step_list = target_foot_step_list[time_idx]

        A_mat = np.eye(12)
        A_mat[0:3, 6:9] = delta_t*np.eye(3)
        A_mat[3:6, 9:12] = delta_t*yaw_mat.T #delta_t*yaw_mat

        B_mat = np.zeros((12, 12))
        for leg_idx in range(num_leg):
            foot_step = foot_step_list[leg_idx] - pos_com
            B_mat[6:9, leg_idx*3:(leg_idx+1)*3] = np.eye(3)*(delta_t/base_mass)
            B_mat[9:12, leg_idx*3:(leg_idx+1)*3] = delta_t*np.matmul(inv_global_inertia, get_cross_product_matrix(foot_step))
        
        jacobian[time_idx*12:(time_idx+1)*12, time_idx*12:(time_idx+1)*12] = -np.eye(12)
        c1, c2, c3, c4 = contact_list[time_idx]
        contact_mat = np.diag([c1]*3 + [c2]*3 + [c3]*3 + [c4]*3)
        jacobian[time_idx*12:(time_idx+1)*12, (time_horizon + time_idx)*12:(time_horizon + time_idx + 1)*12] = np.matmul(B_mat, contact_mat)
        if time_idx > 0:
            jacobian[time_idx*12:(time_idx+1)*12, (time_idx-1)*12:time_idx*12] = A_mat

            temp_mat = np.zeros((12, 12))
            derivative_R = np.array([[-np.sin(yaw), -np.cos(yaw), 0],[np.cos(yaw), -np.sin(yaw), 0],[0, 0, 0]])
            temp_mat[3:6, 5] += delta_t*np.matmul(derivative_R.T, ang_vel_base)
            sum_r_cross_f = np.zeros(3)
            for leg_idx in range(num_leg):
                foot_step = foot_step_list[leg_idx] - pos_com
                sum_r_cross_f += delta_t*np.cross(foot_step, force[leg_idx*3:(leg_idx+1)*3])
            temp_mat[9:12, 5] += np.matmul(derivative_R, np.matmul(inv_base_inertia, np.matmul(yaw_mat.T, sum_r_cross_f)))
            temp_mat[9:12, 5] += np.matmul(yaw_mat, np.matmul(inv_base_inertia, np.matmul(derivative_R.T, sum_r_cross_f)))
            jacobian[time_idx*12:(time_idx+1)*12, (time_idx-1)*12:time_idx*12] += temp_mat

        state = next_state
    return jacobian

In [47]:
init_state_list = np.concatenate([env.model.com_pos, env.model.base_rpy, env.model.base_vel, env.model.base_ang_vel])
init_foot_pos_list = deepcopy(env.model.foot_pos_list)

target_trajectory_list = trajectory_planner.get_target_trajectory_list(vel_cmd, ang_vel_cmd, height_cmd, init_state_list)
contact_list, step_info_list = gait_scheduler.get_contact_list(plan_time_step, plan_time_horizon)
target_foot_step_list = foot_step_planner.get_target_foot_step_list(init_foot_pos_list, step_info_list, target_trajectory_list, vel_cmd, ang_vel_cmd, height_cmd)

In [48]:
args = [init_state_list, target_foot_step_list, contact_list, plan_time_step, base_inertia, base_mass]

init_force_list = np.zeros((plan_time_horizon, 12))
init_x_list = np.concatenate([target_trajectory_list.ravel(), init_force_list.ravel()])

In [49]:
approx_jacobian = np.zeros((12*plan_time_horizon, 24*plan_time_horizon))
eps = 1e-10
func_value = eq_cons_func(init_x_list , args)
for idx in range(len(init_x_list)):
    temp_x_list = np.zeros_like(init_x_list)
    temp_x_list[idx] = eps
    approx_jacobian[:, idx] = (eq_cons_func(init_x_list + temp_x_list, args) - func_value)/eps

In [50]:
jacobian = eq_cons_jacobian(init_x_list, args)

In [51]:
np.sum(abs(jacobian - approx_jacobian) > 1e-5)

0

In [46]:
cnt = 0
mpc_cnt = 0
mpc_period = int(plan_time_step/time_step)
while cnt < mpc_period:
    if mpc_cnt == 0:
        init_state_list = np.concatenate([env.model.com_pos, env.model.base_rpy, env.model.base_vel, env.model.base_ang_vel])
        init_foot_pos_list = deepcopy(env.model.foot_pos_list)

        target_trajectory_list = trajectory_planner.get_target_trajectory_list(vel_cmd, ang_vel_cmd, height_cmd, init_state_list)
        contact_list, step_info_list = gait_scheduler.get_contact_list(plan_time_step, plan_time_horizon)
        target_foot_step_list = foot_step_planner.get_target_foot_step_list(init_foot_pos_list, step_info_list, target_trajectory_list, vel_cmd, ang_vel_cmd, height_cmd)
        state_list, force_list = nlp_solver.get_solution(init_state_list, target_trajectory_list, target_foot_step_list, contact_list)

        '''
        for i in range(plan_time_horizon):
            time.sleep(0.1)
            state = state_list[i]
            pos_base = state[:3]
            rpy_base = state[3:6]
            vel_base = state[6:9]
            ang_vel_base = state[9:]
            orn_base = env.pybullet_client.getQuaternionFromEuler(rpy_base)

            env.pybullet_client.resetBasePositionAndOrientation(cube, pos_base, orn_base)
            env.pybullet_client.resetBaseVelocity(cube, np.zeros(3), np.zeros(3))
            #print(pos_base, target_trajectory_list[i,:3])
        '''

    contact_list, gait_info_list = gait_scheduler.step(time_step)
    inter_weight = 1.0 - float(mpc_cnt)/mpc_period
    inter_force_list = inter_weight*force_list[0] + (1-inter_weight)*force_list[1]
    #print(inter_force_list)
    env.step(inter_force_list, contact_list, desired_foot_list=[])
    time.sleep(time_step)

    mpc_cnt = np.mod(mpc_cnt + 1, mpc_period)
    cnt += 1

Iteration limit exceeded    (Exit mode 9)
            Current function value: 34.9787148966616
            Iterations: 51
            Function evaluations: 488
            Gradient evaluations: 51
